In [1]:
!pip install transformers

In [2]:
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
!nvidia-smi

Tue Dec 19 17:27:47 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# prompt: import drive and mount google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!ls /content/drive/MyDrive/archive.zip

/content/drive/MyDrive/archive.zip


In [ ]:
# prompt: extract /content/drive/MyDrive/archive.zip

!unzip /content/drive/MyDrive/archive.zip


Archive:  /content/drive/MyDrive/archive.zip
replace kaggle_poem_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls

In [ ]:
# prompt: import pandas and load as df /content/drive/MyDrive/kaggle_poem_dataset.csv
import pandas as pd
df = pd.read_csv('kaggle_poem_dataset.csv')


In [ ]:
df.dropna(inplace=True) #remove NA values

In [ ]:
df.head()

In [ ]:
# Preprocess the dataset (e.g., remove NA values, concatenate title and content)
df.dropna(subset=['Content'], inplace=True)  # Ensure no missing content
poems = df['Title'] + "\n\n" + df['Content']  # Concatenate title and content

In [ ]:
print(poems[0])

In [ ]:
from transformers import GPT2Tokenizer
import torch
from torch.utils.data import Dataset, random_split

# Load the GPT tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

class GPT2Dataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length=768):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []

        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

# Create the dataset
dataset = GPT2Dataset(poems, tokenizer, max_length=768)


In [ ]:
from torch.utils.data import DataLoader

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
batch_size = 2  # You can adjust this depending on your GPU memory
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
from transformers import GPT2LMHeadModel, AdamW
from tqdm import tqdm

# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
model.to('cuda')

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Fine-tuning loop
epochs = 5  # You can adjust the number of epochs
for epoch in range(epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        b_input_ids, b_attn_mask = batch
        b_input_ids = b_input_ids.to('cuda')
        b_attn_mask = b_attn_mask.to('cuda')
        model.zero_grad()

        outputs = model(b_input_ids, labels=b_input_ids, attention_mask=b_attn_mask, token_type_ids=None)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch} completed.")


# Save the model
model_save_path = '/content/drive/MyDrive/gpt2_finetuned_poetry.pth'
torch.save(model.state_dict(), model_save_path)


In [ ]:
!find / -name gpt2_finetuned_poetry.pth